## Assignment 18: The goal of this assignment is to balance data to improve a model's balanced acuracy.

### In order to complete the assignment I followed the steps below.

### 1.	The target is Bad. Import the data and correct the type of variables

In [27]:
bank = read.csv('hmeq.csv')
bank$BAD = as.factor(bank$BAD)
names(bank)[1] = "target"

### 2.	Quick Clean the data to handle missing values

In [28]:
quick_clean <- function(df, p)
{
if(p == 1)
{
fixcol = function(y)
{if(is.numeric(y) == TRUE)
{y[is.na(y)] = mean(y, na.rm = TRUE)}
  else
  {
      levels = unique(y)
      y[is.na(y)] = levels[which.max(tabulate(match(y, levels)))]
  }
  return(y)
  }
df = lapply(df, fixcol)
return(data.frame(df))
}
else if (p==0)
{
na.omit(df)
}
  else
  {print("Argument 2 needs to be 0 or 1")}
}

bank = quick_clean(bank, 1)
sum(is.na(bank))

[1] 0

### 3.	Split the data into 70% training, 30% testing

In [29]:
set.seed(42)
library(caret)
splitIndex <- createDataPartition(bank$target, p = .70, list = FALSE, times = 1)
train <- bank[ splitIndex,]
test <- bank[-splitIndex,]

### 4.	Train and test random forest (ranger).  Report the misclassification/accuracy and balanced accuracy. 

In [30]:
set.seed(42)

library(ranger)
model = ranger(target ~., data = train)
pred  = predict(model, data = test)$predictions
cm=confusionMatrix(pred, test$target, positive = "1")
cm$overall['Accuracy']
cm$byClass['Balanced Accuracy']

print("Balanced Accuracy is a good amount less than accuracy due to the data being imbalanced.")

Accuracy 
0.9227756

Balanced Accuracy 
        0.8504935

[1] "Balanced Accuracy is a good amount less than accuracy due to the data being imbalanced."


### 5.	Report the ratio of Default: Non-Default clients. 

In [31]:
table(bank$target)

print("Default: Non-Default ratio")
table(bank$target)[2]/table(bank$target)[1]


   0    1 
4771 1189 

[1] "Default: Non-Default ratio"


1 
0.249214

### 6.	The target is imbalanced.  Thus, it affects the sensitivity, or true positive rate. Follow the below steps to balance the data using undersampling: 

In [32]:
set.seed(42)

#	Split the training data into two dataset:  train1 that contains default clients and train0 containing non-default clients.  Let n1 and n0 be the size of these two data respectively. 


train0 = train[train$target == "0",]
n0 = nrow(train0)

train1 = train[train$target == "1",]
n1 = nrow(train1)

# Select randomly n1 clients from the dataset train0.  (You can use the sample function for this). Let train00 be the dataset containing these clients. 

train00 = train0[sample(1:n0, n1),]

#	Combine (using rbind function) by row train00 and train1 to get dataset train_under

train_under = rbind(train1, train00)

# Train random forest (ranger) on train_under
# Test the model on the test data and report the balanced accuracy.

model = ranger(target ~., data = train_under)
pred  = predict(model, data = test)$predictions
cm=confusionMatrix(pred, test$target, positive = "1")
cm$overall['Accuracy']
cm$byClass['Balanced Accuracy']


print("Overall accuracy was reduced, but balanced accuracy was increased and is just about the same as accuracy.")

Accuracy 
0.8841634

Balanced Accuracy 
         0.880194

[1] "Overall accuracy was reduced, but balanced accuracy was increased and is just about the same as accuracy."


### 7.	Follow the below steps to balance the data using oversampling    

In [33]:
set.seed(42)

# Split the training data into two dataset:  train1 that contains default clients and train0 containing non-default clients.  Let n1 and n0 be the size of these two data respectively.

train0 = train[train$target == "0",]
n0 = nrow(train0)

train1 = train[train$target == "1",]
n1 = nrow(train1)

# Select randomly n0 clients from the dataset train1, with replacement.  (You can use the sample function for this). Let train11 be the dataset containing these clients.

train11 = train1[sample(1:n1, n0, replace = TRUE), ]

# Combine by row train0 and train11 to get dataset train_over

train_over = rbind(train0, train11)

# Train random forest (ranger) on traino_over
# Test the model on the test data and report the balanced accuracy.

model = ranger(target ~., data = train_over)
pred  = predict(model, data = test)$predictions
cm=confusionMatrix(pred, test$target, positive = "1")
cm$overall['Accuracy']
cm$byClass['Balanced Accuracy']

print("Using oversampling, the Balanced Accuracy increases less than when using undersampling, however, the overall accuracy doesn't decrease as much.")

Accuracy 
0.9199776

Balanced Accuracy 
        0.8698482

[1] "Using oversampling, the Balanced Accuracy increases less than when using undersampling, however, the overall accuracy doesn't decrease as much."


### 8.	Write a function called Quick_bal.  This function... Inputs:  a dataset argument with a target variable named target; a method argument specifying undersampling (1) or oversampling (2).	Outputs: a dataset with a balanced target. 

In [34]:
Quick_bal = function(df, method)
{
  if(method == 1) #Undersampling
  {
balance0 = df[df$target == "0",]
n0 = nrow(balance0)

balance1 = df[df$target == "1",]
n1 = nrow(balance1)

balance00 = balance0[sample(1:n0, n1),]

balanced_under = rbind(balance1, balance00)
print("Undersampling positive and and negative target group sizes")
table(balanced_under$target)
  }
  else
  if(method == 2) #Oversampling
  {
balance0 = df[df$target == "0",]
n0 = nrow(balance0)

balance1 = df[df$target == "1",]
n1 = nrow(balance1)

balance11 = balance1[sample(1:n1, n0, replace = TRUE),]

balanced_over = rbind(balance11, balance0)
print("Oversampling positive and and negative target group sizes")
table(balanced_over$target)
  }
}

Quick_bal(bank, 1)
Quick_bal(bank, 2)

[1] "Undersampling positive and and negative target group sizes"



   0    1 
1189 1189 

[1] "Oversampling positive and and negative target group sizes"



   0    1 
4771 4771 

### 9.	Put 6 and 7 into a function called Quick_model_bal. This function, Inputs: a dataset with a target variable named target, Output: the balanced accuracies of random forest with undersampling and oversampling being applied on the training dataset. 

In [35]:
Quick_model_bal = function(df)
{
set.seed(42)
library(caret)
splitIndex <- createDataPartition(df$target, p = .70, list = FALSE, times = 1)
ftrain <- df[ splitIndex,]
ftest <- df[-splitIndex,]

set.seed(42)
u_train0 = ftrain[ftrain$target == "0",]
u_n0 = nrow(u_train0)
u_train1 = ftrain[train$target == "1",]
u_n1 = nrow(u_train1)
u_train00 = u_train0[sample(1:u_n0, u_n1),]
train_under = rbind(u_train1, u_train00)
u_model = ranger(target ~., data = train_under)
u_pred  = predict(u_model, data = ftest)$predictions
u_cm=confusionMatrix(u_pred, ftest$target, positive = "1")

print("Undersampling Accuracy and Balanced Accuracy")
print(u_cm$overall['Accuracy'])
print(u_cm$byClass['Balanced Accuracy'])

set.seed(42)
o_train0 = ftrain[ftrain$target == "0",]
o_n0 = nrow(o_train0)
o_train1 = ftrain[ftrain$target == "1",]
o_n1 = nrow(o_train1)
o_train11 = o_train1[sample(1:o_n1, o_n0, replace = TRUE), ]
train_over = rbind(o_train0, o_train11)
o_model = ranger(target ~., data = train_over)
o_pred  = predict(o_model, data = ftest)$predictions
o_cm=confusionMatrix(o_pred, ftest$target, positive = "1")
print("Oversampling Accuracy and Balanced Accuracy")
print(o_cm$overall['Accuracy'])
print(o_cm$byClass['Balanced Accuracy'])
}


Quick_model_bal(bank)

[1] "Undersampling Accuracy and Balanced Accuracy"
 Accuracy 
0.8841634 
Balanced Accuracy 
         0.880194 
[1] "Oversampling Accuracy and Balanced Accuracy"
 Accuracy 
0.9199776 
Balanced Accuracy 
        0.8698482 
